# COVID Troll QA (Thai)

A tutorial to create a Question Answering system to answer COVID-19 related questions.

## Data

From this page https://nlpforthai.com/tasks/question-answering/ , `iapp-wiki-qa-dataset` and `Thai WIKI QA` are used in this tutorial.

Big thanks to all parties!


## Steps in a nutshell
0. Load data and convert to `transformers` package format
1. Load a pretrained language model (wangchanBERTa)
2. Fine-tune the model in step 1.
3. Create a document ranking logic to retrive several relevant documents (BM25)
4. Find answer candidates in the documents in step 3.

## 0. Load data and convert to `transformers` package format

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
prechecked_broken_pos = [11860, 12204, 12228, 12559, 12952, 12965, 13612, 13851, 13893, 14028, 14086, 14166, 14815, 15570, 15579, 15674, 15675, 15787, 16209, 16332, 16376, 16427]

In [4]:
df_qa = pd.read_csv('Thai_QA_NSCWiki_iApp.csv')

# keep only rows that have an answer before 512nd position
max_len = 500
print("Before drop:", len(df_qa))
df_qa = df_qa[df_qa['answer_end_pos'] < max_len - 30].reset_index(drop=True)
df_qa = df_qa.drop(index=prechecked_broken_pos).reset_index(drop=True)
print("After drop:", len(df_qa))

Before drop: 22181
After drop: 16950


In [9]:
df_qa.head(2)

,question_id,question,answer,answer_begin_pos,answer_end_pos,context,article_id,source
0,1,สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง Ben...,ฮิกกิ้นส์,447,456,เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู...,115035,thai_wiki_qa
1,2,ลูนา 1 เป็นยานอวกาศลำแรกในโครงการลูนาของโซเวีย...,เมชตา,57,62,ลูนา 1 ลูนา 1 (อี-1 ซีรีส์) ซึ่งในขณะนั้นรู้จั...,376583,thai_wiki_qa


In [10]:
train_contexts = df_qa['context'].apply(lambda x: x.lower()).tolist()
train_questions = df_qa['question'].apply(lambda x: x.lower()).tolist()
train_answers = [
                 {'answer_start': row['answer_begin_pos'], 
                  'answer_end': row['answer_end_pos'], 
                  'text': row['answer'].lower()
                  } for _, row in df_qa.iterrows()
                  ]

In [11]:
train_answers[2]

{'answer_end': 199, 'answer_start': 192, 'text': 'ปี 1933'}

## 1. Load a pretrained language model (wangchanBERTa)

In [12]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "airesearch/wangchanberta-base-att-spm-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForQuestionAnswering: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['qa_outputs.bias', 

In [13]:
%%time
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True, max_length=max_len)

CPU times: user 1min 23s, sys: 1.52 s, total: 1min 24s
Wall time: 43.5 s


In [14]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    broken_pos = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            print("Start position of", i, "is broken")
            broken_pos.append(i)
            start_positions[-1] = tokenizer.model_max_length            
        if end_positions[-1] is None:
            print("End position of", i, "is broken")
            end_positions[-1] = tokenizer.model_max_length
            broken_pos.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return broken_pos

broken_pos = add_token_positions(train_encodings, train_answers)

In [15]:
import torch

class WikiQATHDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):        
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = WikiQATHDataset(train_encodings)

## 2. Fine-tune the model in step 1.

In [17]:
%%time

from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = 'cpu'
print("Device:", device)

model.to(device)
model.train()

losses = []

batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

optim = AdamW(model.parameters(), lr=2e-5)

for epoch in range(4):
    
    loss_ep = []
    for i, batch in enumerate(train_loader):
        if i % 500 == 0:
          print("Batch", i)
        optim.zero_grad()        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        try:
          outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
          loss = outputs[0]
          loss_ep.append(loss.item())
          loss.backward()
          optim.step()
        except RuntimeError as e:
          print("Batch error", i, e)

    loss_total = sum(loss_ep)
    losses.append(loss_total)
    
    print("Epoch:", epoch + 1, "Loss:", loss_total)
    # save model to google drive
    print("Save model to google drive")
    model.save_pretrained('model_checkpoint/')
    # os.system('cp model_checkpoint/* drive/MyDrive/ThaiWikiQA/model_checkpoint')


_ = model.eval()

Device: cuda
Batch 0
Batch 500
Batch 1000
Batch 1500
Batch 2000
Epoch: 1 Loss: 2080.4841587916017
Save model to google drive
Batch 0
Batch 500
Batch 1000
Batch 1500
Batch 2000
Epoch: 2 Loss: 1459.9210587069392
Save model to google drive
Batch 0
Batch 500
Batch 1000
Batch 1500
Batch 2000
Epoch: 3 Loss: 1065.663506789133
Save model to google drive
Batch 0
Batch 500
Batch 1000
Batch 1500
Batch 2000
Epoch: 4 Loss: 864.8384659551084
Save model to google drive
CPU times: user 2h 2min 45s, sys: 22.4 s, total: 2h 3min 7s
Wall time: 2h 3min 29s


In [64]:
text = r""" กรุงเทพมหานคร เป็นเมืองหลวงและนครที่มีประชากรมากที่สุดของประเทศไทย เป็นศูนย์กลางการปกครอง การศึกษา การคมนาคมขนส่ง การเงินการธนาคาร การพาณิชย์ การสื่อสาร และความเจริญของประเทศ เป็นเมืองที่มีชื่อยาวที่สุดในโลก ตั้งอยู่บนสามเหลี่ยมปากแม่น้ำเจ้าพระยา มีแม่น้ำเจ้าพระยาไหลผ่านและแบ่งเมืองออกเป็น 2 ฝั่ง คือ ฝั่งพระนครและฝั่งธนบุรี กรุงเทพมหานครมีพื้นที่ทั้งหมด 1,568.737 ตร.กม. มีประชากรตามทะเบียนราษฎรกว่า 5 ล้านคน ทำให้กรุงเทพมหานครเป็นเอกนคร (Primate City) จัด มีผู้กล่าวว่า กรุงเทพมหานครเป็น 'เอกนครที่สุดในโลก' เพราะมีประชากรมากกว่านครที่มีประชากรมากเป็นอันดับ 2 ถึง 40 เท่า[3]"""

In [65]:
questions = ["แม่น้ำเจ้าพระยา ไหลผ่านและแบ่งเมืองออกเป็นฝั่งอะไร"]

In [20]:
_ = model.to('cpu')

In [21]:
import numpy as np

def norm_with_softmax(inputs, x):
    x = x.detach().numpy()
    
    # remove non-context part    
    p_mask = [tok != 1 for tok in inputs.sequence_ids(0)]
    
    undesired_tokens = np.abs(np.array(p_mask) - 1) & inputs['attention_mask'].detach().numpy()
    undesired_tokens_mask = undesired_tokens == 0.0
    
    x = np.where(undesired_tokens_mask, -10000.0, x)
    
    # calculate softmax
    sm = np.exp(x - np.log(np.sum(np.exp(x), axis=-1, keepdims=True)))
    return sm

In [62]:
def get_answer_and_score(context, question, print_qa=False):
    model.eval()
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, 
                                   return_tensors="pt", truncation=True, 
                                   max_length=max_len)
    
    input_ids = inputs["input_ids"].tolist()[0]
    

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    qa_output = model(**inputs)
    answer_start_scores = norm_with_softmax(inputs, qa_output['start_logits'])[0]
    answer_end_scores = norm_with_softmax(inputs, qa_output['end_logits'])[0]

    answer_start = np.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = np.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score    
    score = answer_start_scores[answer_start] * answer_end_scores[answer_end]
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    if print_qa:
      print(f"Question: {question}")
      print(f"Answer: {answer}\n")
      
    return answer, score

In [66]:
get_answer_and_score(text, questions[0])

('ฝั่งพระนครและฝั่งธนบุรี', 0.0051432136)

## 3. Create a document ranking logic to retrive several relevant documents (BM25)

In [25]:
import glob

In [26]:
from rank_bm25 import BM25Okapi

In [29]:
from pythainlp.tokenize import word_tokenize

In [30]:
doc_dir = 'line_grp_articles/'

In [71]:
def chunk(doc, chunk_max_char=800):
    doc_len = len(doc)
    n_chunks = int(np.ceil(doc_len / chunk_max_char))
    
    chunks = []
    for i in range(n_chunks):
        begin = chunk_max_char * i
        end = begin + chunk_max_char
        chunks.append(doc[begin:end])
    return chunks

def chunk_docs(docs, chunk_max_char=800):    
    doc_chunks = []
    for doc in docs:
        chunks = _chunk(doc, chunk_max_char)
        doc_chunks.extend(chunks)
    
    return doc_chunks        

In [72]:
filenames = glob.glob(doc_dir + '*.txt')

corpus = []
corpus_tkn = []

for fn in filenames:
    with open(fn, 'r') as f:
        doc = f.read().replace('\n', '  ')   # remove linebreak
        doc_chunks = chunk(doc)
        corpus.extend(doc_chunks)   

corpus_tkn = list(map(word_tokenize, corpus))

In [73]:
bm25 = BM25Okapi(corpus_tkn)

In [74]:
def get_best_matches(query, n=3):
    query_tkn = word_tokenize(query)
    docs = bm25.get_top_n(query_tkn, corpus, n=n)
    return docs

In [115]:
def ask_model(query, n=3, chunk_max_char=800):
    # retrive most relevant n*1.5 documents (generate many candidates, then filter blank answers out)
    docs = get_best_matches(query, n=n)
    # chunk extra-long docs
    doc_chunks = chunk_docs(docs, chunk_max_char)
    
    # ask q question on each chunk
    ans_and_scores = sorted([get_answer_and_score(doc, query) for doc in doc_chunks], key=lambda x: x[1], reverse=True)
    # remove blank answers
    ans_and_scores = list(filter(lambda x: len(x[0]) > 0, ans_and_scores))
    # get most confident one
    best_ans = ans_and_scores[0]
    
    return best_ans, ans_and_scores        

## 4. Find answer candidates in the documents in step 3.

In [102]:
ask_model("หอมแดง ควรกินกับอะไร เพื่อรักษาโควิด", n=2)

(('หอมแดง และ กระเทียม', 0.00096342963),
 [('หอมแดง และ กระเทียม', 0.00096342963), ('แอสไพริน', 0.00019585372)])

In [103]:
ask_model("วัคซีนอะไรดีที่สุด", n=2, chunk_max_char=600)

(('โปรตีน', 0.0042575966),
 [('โปรตีน', 0.0042575966),
  ('ดร.อาทิตย์” โพสต์ สหรัฐฯยอมรับแล้ว วัคซีนที่ดีที่สุดคือของจีน',
   0.0011938916)])

In [104]:
ask_model("ติดโควิดควรกินอะไร", n=2)

(('กินกระท่อม', 0.015897416),
 [('กินกระท่อม', 0.015897416), ('กินยาจีน', 0.004160854)])

In [105]:
ask_model("ติดโควิดห้ามกินอะไร", n=2)

(('กินยาจีน', 0.0019697412),
 [('กินยาจีน', 0.0019697412), ('ห้ามกินทุเรียน', 0.0012407483)])

In [106]:
ask_model("ใครปิดทองหลังวัคซีน", n=2)

(('จะไปจับความผิดพลาดได้ไหม', 0.04440104),
 [('จะไปจับความผิดพลาดได้ไหม', 0.04440104),
  ('ไทยจะลดความเสี่ยง ที่เพื่อนบ้านจะนําเชื้อเข้ามาแพร่อีกด้วย ย้ําอีกครั้ง <unk> เลือกไทย ไม่ใช่เพราะว่า ไทยน่าสนใจและอยากเข้ามาเปิดประมูล แต่เป็นเพราะความร่วมมือภาคเอกชน ที่อาศัยเส้นสายจนเขายอมรับ และยอมเสียผลประโยชน์บางอย่าง เพื่อทําให้อุดมการณ์ของ <unk>xford เป็นจริง นั่นคือ การกระจายวัคซีนให้มากที่สุด และถูกที่สุดเท่าที่จะทําได้',
   0.00034202)])

In [114]:
ask_model("SCG ทำอะไร", n=5)

(('ที่ประเทศกําลังพัฒนา ขนาดกลางอย่างไทย', 0.02846697),
 [('ที่ประเทศกําลังพัฒนา ขนาดกลางอย่างไทย', 0.02846697),
  ('ยุววิศวกรบพิธ', 0.021682635),
  ('ศ.นพ.ยง ภู่วรวรรณ', 0.010183519),
  ('ตลาดวัคซีนเป็นของผู้ขาย ไม่ใช่ผู้ซื้อ', 0.0007883236),
  ('ใช้งบ <unk> 100 ล้าน พร้อมพนักงาน 50-60 คน เข้าไปรุมช่วยสยามไบโอไซ',
   0.00011320774)])

In [111]:
ask_model("สหรัฐบอกว่า pfizer เป็นวัคซีนอันดับที่เท่าไหร่", n=1)

(('ดร.อาทิตย์” โพสต์ สหรัฐฯยอมรับแล้ว วัคซีนที่ดีที่สุดคือของจีน ระบุ “ไฟเซอร์” อยู่อันดับที่ 6',
  0.0016930654),
 [('ดร.อาทิตย์” โพสต์ สหรัฐฯยอมรับแล้ว วัคซีนที่ดีที่สุดคือของจีน ระบุ “ไฟเซอร์” อยู่อันดับที่ 6',
   0.0016930654)])

In [89]:
ask_model("วัคซีนอะไรดีที่สุด", n=1)

(('ดร.อาทิตย์” โพสต์ สหรัฐฯยอมรับแล้ว วัคซีนที่ดีที่สุดคือของจีน',
  0.0011938916),
 [('ดร.อาทิตย์” โพสต์ สหรัฐฯยอมรับแล้ว วัคซีนที่ดีที่สุดคือของจีน',
   0.0011938916)])

In [97]:
ask_model("นักศึกษาอินเดียค้นพบอะไร", n=3)

(('ยาฆ่าเชื้อโควิด 19', 0.0077594207),
 [('ยาฆ่าเชื้อโควิด 19', 0.0077594207), ('พริกไทยบ่น 1 ช้อนชา', 0.0014260751)])

In [117]:
ask_model("วัคซีน mRNA จะเข้าไปทำอะไรร่างกาย", n=5)

(('จะมีผลข้างเคียงใดๆในระยะสั้นหรือระยะยาว', 0.00424137),
 [('จะมีผลข้างเคียงใดๆในระยะสั้นหรือระยะยาว', 0.00424137),
  ('ติดเชื้อโควิดไวรัสจริงเข้ามา', 0.0024261223),
  ('ประวัติศาสตร์', 0.0022650147)])